# Импорт библиотек

In [1]:
import os
import pandas as pd
import glob
import string
import nltk
import re
import numpy as np

# Отсев невалидных данных и определение валидных


In [ ]:
class Utils():
    """
    классе происходит отсеивание невалидных документов,
    которые не удалось адекватно прочитать.

    На выходе получаем список валидно прочитанных файлов
    """
    
    root_directory = os.chdir('/content/drive/MyDrive/parser')# input path
    list_of_files = glob.glob('**/*.txt', recursive=True)
    invalid_parsing = []
    trash = ['Волковыск-1.txt','Волковыск.txt','Вороново-1.txt',
         'Вороново.txt', 'Черновик-1.txt', 'Черновик.txt']
    for file in list_of_files:
        try:
             with open(file, encoding='utf-8') as f:
                    num = f.readline()
                    file_ = f.read()
                    if num and file_:
                        pass
        except:
            invalid_parsing.append(file)
    for i in invalid_parsing:
        list_of_files.remove(i)
    for i in trash: 
        list_of_files.remove(i)


list_of_files = Utils.list_of_files


# Определяем пустые списки, которые будут наполняться данными


In [ ]:
plt_ = [] # Список для показателя "Тромбоциты"
wbc_ = [] # Список для "Лейкоцитов"
hgb_ = [] #Список для "Гемоглобина"
crea_ = [] #Список для "Креатинина"
ldh_ = []#Список для "Лактатдегидрогеназы"
sex_ = []#Список для "Пола"
crp_ = []#Список для "С-реактивного протеина"
tr = []#Список для "Лечения"
bir = []#Список для "Даты рождения"
ad = []#Список для "Даты поступления"
dis = []#Список для "Даты выписки"
ID_ = []#Список для "№ эпикриза"
dgs_ = []#Список для "Диагноза"
rf_ = []#Список для "Дыхательный недостаточности"
alt_ = []#Список для "Аланинтрансаминазы"
ast_ = []#Список для "Аспартаттрансаминазы"
pct_ = []#Список для "Прокальцитонина"
esr_ = []#Список для "Скорости оседания эритроцитов"
fibrin_ = []#Список для "Фибриногена"

# Пример эпикриза
ВЫПИСНОЙ ЭПИКРИЗ № **170**
Находился(ась) в пульмонологическом отделении ГОКБ МР
Ф.И.О: 
Дата рождения:**02.08.1966**
Адрес регистрации: 
Адрес проживания: 
Место работы:
Должность:
Поступил: **13.01.2021**
Выписан: **26.01.2021**

Диагноз клинический: **Тяжёлая коронавирусная (ТОРС-КОВ-2 ПЦР+ от 12.01.2021 ) инфекция. Острое вирусное повреждение лёгких, КТ 3 ДН 1 ст. **

Диагноз заключительный клинический:  **B34.2  / (основной) /   Тяжёлая коронавирусная (ТОРС-КОВ-2 ПЦР+ от 12.01.2021) инфекция. Острое вирусное повреждение лёгких, КТ 3 в стадию обратного развития, ДН 0 ст. ИБС: Атеросклеротический кардиослкероз. Атеросклероз аорты, коронарных артерий. АГ 2ст., риск 4. Н 1ст. Сахарный диабет, тип 2, клинико-метаболическая субкомпенсация. МКБ ( оперативное лечение 2019) ХБП С2 ( рСКФ -72 мл/мин/1,73м2). Дискогенная радикулопатия S1 слева, стойкий выраженный болевой синдром. Диабетическая полинейропатия, дистальный тип, с умеренными сенсо-моторными нарушениями. Ожирение 3 ст. (ИМТ - 42 кг/м2) **

Консультации врачей-узких специалистов, консилиумы врачей:  
Консультация невролога 14.01.2021, 20.01.2021
Консультация нейрохирурга 20.01.2021 показаний к экстренному оперативному вмешательству нет

Результаты инструментальных и аппаратных методов исследования:    
ЭКГ от 13.01.2021 Ритм синусовый, правильный, ЧСС - 93 уд в мин, норм.полож.ЭОС, неспецифические изменения в миокарде нижней стенки ЛЖ
МРТ от 19.01.2021 Заключение: МР-картина дегенеративно-дистрофических изменений пояснично-крестцового отдела позвоночника в виде остеохондроза с экструзией диска в Л5-С1 сегменте
Результаты лабораторных исследований:
Гликемический профиль 14.01.2021: 8 ч. 00 мин. 25.8 ммоль/л ; 12 ч. 00 мин. 24.3 ммоль/л ; 16 ч. 00 мин. 25.5 ммоль/л ; 
Гликемический профиль 20.01.2021: 8 ч. 00 мин. 9.8 ммоль/л ; 12 ч. 00 мин. 10.3 ммоль/л ; 16 ч. 00 мин. 9.5 ммоль/л ; 
Гемостазиограмма 14.01.2021: Активированное частичное тромбопластиновое время 22.1 с ; Ratio АЧТВ 0.744  ; Протромбированное время 11.2 с ; Протромбиновый индекс 1.01  ; Международное нормализованное отношение 0.981  ; **Фибриноген 8.1** г/л ; 
Биохимическое исследование крови 14.01.2021: Общий белок 71 г/л ; Мочевина 10.7 ммоль/л ; Креатинин 98.1 мкмоль/л ; C-реактивный белок **41.5** мг/л ; Билирубин общий 10.5 мкмоль/л ; Аспартатаминотрансфераза 27.8 Ед/л ; Аланинаминотрансфераза 54.0 Ед/л ; Натрий 138.7 ммоль/л ; Калий 5.89 ммоль/л ; э\
**Прокальцитонин от 14.01.2021 0.14 нг/мл**
Биохимическое исследование крови 15.01.2021 Глюкоза - 22,1 ммоль/л
Общий анализ крови 14.01.2021: WBC (Лейкоциты) **9.1** 10*9/l ; RBC (Эритроциты) 4.09 10*12/l ; HGB (Гемоглобин) 129 g/l ; PLT (тромбоциты) 495 10*9/l ; СОЭ 35 мм/ч ; Нейтрофилы палочкоядерные 3 % ; Нейтрофилы сегментоядерные 68 % ; Лимфоциты 26 % ; Моноциты 3 % ; \С у м м а   к л е т о к: 100  ;  
Прокальцитонин 0.11 нг
Состояние при выписке: относительно удовлетворительное. При выписке сатурация крови кислородом при дыхании атмосферным воздухом - 97%, ЧД 18 в мин. уровень лейкоцитов более 3 тыс. на мл., не лихорадит более 3 дней без жаропонижающих. Уровень СРБ менее 2х норм

**Проведенное лечение: гепарин, пантопрозол, ацецезон, гликлазид, метформин, лизиноприл, метопролол, индапамид, эуфиллин, новокаин, димедрол, вит группы В, ксефокам, толперизон, карбомазепин, кетаролак, зопиклон, пентоксифиллин, трамадол, мелоксикам, ФТЛ**

Рекомендации:Наблюдение участкового терапевта, эндокринолога, невролога по месту жительства (Д3)
Избегать переохлаждений, витаминно-минеральный комплекс
Снижение употребления поваренной соли менее 5грамм в сутки, животных жиров, рафинированных углеводов. Повышение употребления овощей , омега-3 ненасышеные ЖК, умеренные аэробные физические нагрузки
Контроль гликированного гемоглобина
Целевой уровень ЛПНП - ниже 1,8 ммоль/л
Ривароксабан 10 мг 1 раз в день ( 45 дней), при невозможности - аспикард 75 мг после обеда ( не менее 3 месяцев) 
Ацетилцистеин 1,2 в день - 45 дней
Гликлазид 60мг 1 табл. утром
Метформин 1000мг 2 раза в день
Лизиноприл/гидрохлортиазид 10/12,5мг утром
Метопролол 25  мг 2 раза в день
Невролог:
1) Дексамин 2,0 в/м № 5
2) Нимесулид 100 мг 2 раза  в день ( утро, вечер) при необходимости
3) Мидокалм 150 мг по 1 табл 3 раза в день - 14 дней
4) Боривит 2,0 в/м № 10



Таким образом, надо "достать" интересующие нас данные

# Методы, которые позволяют получить данные

In [ ]:
def esr(file_):
  file_ = ''.join(file_.split()).lower()
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  patt = re.compile(r'(?<=соэ)\d+')
  patt1 = patt.findall(file_)
  try:
    if patt1:
      return np.float_(patt1)
    else:
      return 'None'
  except:
    pass

def fibrn(file_):
  file_ = ''.join(file_.split()).lower()
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r',', '.', file_)
  patt = re.compile(r'(?:фибриноген|фибр\.|фибрин|ф)(\d+.\d+)')
  patt1 = patt.findall(file_)
  try:
    if patt1:
      return np.float_(patt1)
    else:
      return 'None'
  except:
    pass


def ID(num):
  """returns ID of the patient"""
  for i in num.split():
    for char in i.split():
        if char.isnumeric():
            try:
                return int(char)
            except:
                return 'None'
            
            
def treatment(file_):
  """ returns srting of remedies used for the patient """
  for line in file_.split('\n'):
    if re.search('Провед.нное.лечение', line):  
        return(line.replace('Проведенное лечение:', \
                            '').replace('ЛФК', '').replace('ФТЛ', ''))
    

def born_adm_disch(file_):
    """ finds all nessesary dates in the epicrisis"""
    pattern = re.compile("(\d{2}).(\d{2}).(\d{4})") # check all dates
    birthday, admission = pattern.findall(file_)[:2]
    discharging = pattern.findall(file_)[-1]
    try:
        return ':'.join(birthday), ':'.join(admission), ':'.join(discharging)
    except:
        return 'None'


pattern_diagnosis = ['Диагноз:','Диагноз клинический:','Диагноз', 'Д-з:',
                     "Диагноз заключительный:"]
def diagnosis_dirty(file_):
  int_ = 0
  for i in file_.split():
    int_ += 1
    if i in pattern_diagnosis:
      return file_.split()[int_+1:int_+40]    
  

def crp(file_):
  """returns all crp values"""
  file_= ''.join(file_.split())
  file_ = re.sub(r'[ЦСC]РБ|\w\D(реактив.|реакт.|реак.)белок', 'С-реактивныйбелок', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'(?<!С-реактивныйбелок)\d\d\.\d\d', '', file_)
  file_ = re.sub(r',', '.', file_)
  pattern_1 = re.compile(r'(?:\w\Dреактивныйбелок|\w\Dреактивныйбелокдо)(\d*\.\d+|\d+)')
  pattern_2 = pattern_1.findall(file_)
  try:
    if pattern_2:
      return np.float_(pattern_2)
    else:
      return 'None'
  except:
    pass


def gender(file_):
  file_ = file_.title()
  file_ = ''.join(file_.split())
  file_ = re.sub(r'Диагноз.*', '', file_)
  patt1 = re.compile(r'[А-Я](\w{,19}(\w|\ич))[А-Я]\w{,20}(вна)')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return 'female'
    else:
      return 'male'
  except:
    pass
  

def ldh(file_):
  file_= ''.join(file_.split())
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'\wактатдегидрогеназ\w', 'лдг', file_)
  file_ = re.sub(r'(?<!лдг)\d\d\.\d\d', '', file_)
  patt1 = re.compile(r'(?<=лдг)\d{,4}')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return np.float_(patt2)
    else:
      return 'None'
  except:
    pass



def cre(file_):
  """Returns all creatinine values"""
  file_= ''.join(file_.split())  
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'(\wреатини\w|креатин|креат)(?![а-яА-Я,])', 'cre', file_)
  file_ = re.sub(r'(?<!cre)\d\d\.\d\d', '', file_)
  patt1 = re.compile(r'(?<=cre)\d{2,3}')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return np.float_(patt2)
    else:
      return 'None'
  except:
    pass


def hgb(file_):
  """returns list of str with level of hemoglobin"""
  file_= ''.join(file_.split())  
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  patt = re.compile(r'гемоглобин|гемогл|гб|гем')
  patt1 = re.sub(patt, r'hgb', file_)
  patt2 = re.compile(r'(?<=hgb)\d{2,3}')
  all_hgb = patt2.findall(patt1)
  try:
    if all_hgb:
      return np.float_(all_hgb)
    else:
      return 'None'
  except:
    pass


def wbc(file_):
  """returns list of str with level of wbc"""
  file_= ''.join(file_.split())
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'10(\*|[еe])9', '', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'\Dбщийанализкрови', 'оак', file_)
  file_ = re.sub(r'(?<=оак)гб\d*,', '', file_)
  file_ = re.sub(r',', '.', file_)
  patt1 = re.compile(r'(?<=оак|wbc)(?:л|лейкоцит\w)(\d*\.\d+|\d+)')
  all_wbc = patt1.findall(file_)
  try:
    if all_wbc:
      return np.float_(all_wbc)
    else:
      return 'None'
  except:
    pass


def plt(file_):
  """returns list of str with level of plt"""
  file_= ''.join(file_.split())
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'10(\*|[еe])9', '', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  patt1 = re.compile(r'\wромбоцит\w|(?<![а-яА-Я])тр(?![а-яА-Я])')
  file_ = re.sub(patt1, r'plt', file_)
  file_ = re.sub(r'(?<!plt)\d\d\.\d\d', '', file_)
  patt2 = re.compile(r'(?<=plt)\d{3}')
  all_plt = patt2.findall(file_)
  try:
    if all_plt:
      return np.float_(all_plt)
    else:
      return 'None'
  except:
    pass


def rf(file_):
  file_= ''.join(file_.split())  
  file_ = re.sub(r'[а-яА-ЯёЁ]ДН[^Шш0-9]', 'ДН'.lower(), file_)
  file_ = re.sub(r'(?<=ДН)[1iI]{3}|[Шш]', '3', file_)
  file_ = re.sub(r'(?<=ДН)[1iI]{2}|1-2', '2', file_)
  file_ = re.sub(r'(?<=ДН)[оoОO]', '0', file_)
  patt1 = re.compile(r'(?<=ДН)\d')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return patt2
    else:
      return 0
  except:
      return 0



def alt(file_):
  file_= ''.join(file_.split())
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'\wланинаминотрансфераз\w|алат', 'алт', file_)
  file_ = re.sub(r'(?<!алт)\d\d\.\d\d', '', file_)
  file_ = re.sub(r',', '.', file_) 
  patt1 = re.compile(r'(?<=алт)(\d*\.\d+|\d+)')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return np.float_(patt2)
    else:
      return 'None'
  except:
    pass


def ast(file_):
  file_= ''.join(file_.split())
  file_ = file_.lower()
  file_ = re.sub(r'[():]', '', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}', '', file_)
  file_ = re.sub(r'\wспартатаминотрансфераз\w|асат', 'аст', file_)
  file_ = re.sub(r'(?<!аст)\d\d\.\d\d', '', file_)
  file_ = re.sub(r',', '.', file_)
  patt1 = re.compile(r'(?<=аст)(\d*\.\d+|\d+)')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return np.float_(patt2)
    else:
      return 'None'
  except:
    pass


def pct(file_):
  file_= ''.join(file_.split())
  file_ = file_.lower()
  file_ = re.sub(r'(?<=\,\d)\,|\.\d{2}\.\d\d\.\d{2,4}.', '', file_)
  file_ = re.sub(r'\d{,2}\.\d\d\.\d{2,4}(\.|)', '', file_)
  file_ = re.sub(r'[–-]|[():<>=]', '', file_)
  file_ = re.sub(r'менее', '', file_)
  file_ = re.sub(r'от', '', file_)
  file_ = re.sub(r'(\wрокальцитони\w|прокальцитон|прокальцит|\
                  прокальц|рст)(?![а-яА-Я,])', 'pct', file_)
  file_ = re.sub(r',', '.', file_)
  patt1 = re.compile(r'(?<=pct)(\d*\.\d+|\d+)')
  patt2 = patt1.findall(file_)
  try:
    if patt2:
      return np.float_(patt2)
    else:
      return 'None'
  except:
    pass

# Запускаем методы

In [ ]:
for file in list_of_files:
    try:
         with open(file, encoding='utf-8') as f:
                num = f.readline()
                file_ = f.read()
                ID_.append(ID((num)))
                tr.append(treatment((file_)))
                bir.append(born_adm_disch(file_)[0])
                ad.append(born_adm_disch(file_)[1])
                dis.append(born_adm_disch(file_)[2])
                dgs_.append(diagnosis_dirty((file_)))
                crp_.append(crp(file_))
                sex_.append(gender(file_))
                ldh_.append(ldh(file_))
                crea_.append(cre(file_))
                hgb_.append(hgb(file_))
                wbc_.append(wbc(file_))
                plt_.append(plt(file_))
                rf_.append(rf(file_))
                alt_.append(alt(file_))
                ast_.append(ast(file_))
                pct_.append(pct(file_))
                esr_.append(esr(file_))
                fibrin_.append(fibrn(file_))
    except:
        print('stp')

# !!! Работа со списками
## 1) Начнём с диагнозов

In [ ]:
diagnosis_cleaned =  []
for diagnos in dgs_:
    try:
        diagnosis_cleaned.append(' '.join(diagnos))
    except:
        diagnosis_cleaned.append('None')
        
        
        
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))





list_wo_punct = []
for sent in diagnosis_cleaned:
    try:
        sent = remove_punctuation(sent.lower())
        list_wo_punct.append(sent)
    except:
        pass
        


Таким образом, мы имеет список со строками без пунктуции, далее, нам надо найти ключевые слова, чтобы определить наличие или отсутствие конкретных заболеваний

# # - поиск коронавирусной инфекции. 
- если есть коронавирусная инфекция - 1, иначе - 0

In [ ]:
covid_list = []    
for sent in list_wo_punct:
    pattern = re.sub(r'covid19', 'covid', sent )
    pattern = re.sub(r'коронавирусная', 'covid', pattern)
    pattern = re.sub(r'короновирусная', 'covid', pattern)
    pattern = re.sub(r'b342', 'covid', pattern)
    pattern = re.sub(r'sarscov2', 'covid', pattern)
    pattern = re.sub(r'торсков2', 'covid', pattern)
    pattern_1 = re.compile(r'covid')
    patt_2 = re.search(pattern_1, pattern)
    try:
        covid_list.append(patt_2)
    except:
        pass
    
    
c =[]
#если есть коронавирусная инфекция - 1, иначе - 0
for i in covid_list:
    if not i:
        c.append(0)
    else:
        c.append(1) 

# # - поиск пневмоний

In [ ]:
pn_list = []
for sent in list_wo_punct:
    pattern = re.sub(r'пневмония', 'J18', sent )
    pattern = re.sub(r'пне...ния', 'J18', pattern)
    pattern = re.sub(r'внегоспитальная', 'J18', pattern)
    pattern = re.sub(r'внебольничная', 'J18', pattern)
    pattern_1 = re.compile(r'J18')
    patt_2 = re.search(pattern_1, pattern)
    try:
        pn_list.append(patt_2)
    except:
        pass  
    
    
p = []
# если есть пневмония - 1, иначе - 0
for i in pn_list:
    if not i:
        p.append(0)
    else:
        p.append(1)

## Поиск ишемической болезни сердца

In [ ]:
ihd_list = []
for sent in list_wo_punct:
    pattern = re.sub(r'ссн', 'ИБС', sent )
    pattern = re.sub(r'ибс', 'ИБС', pattern)
    pattern = re.sub(r'атеросклеротический', 'ИБС', pattern)
    pattern = re.sub(r'кардиосклероз', 'ИБС', pattern)
    pattern = re.sub(r'фп', 'ИБС', pattern)
    pattern = re.sub(r'фибрил....', 'ИБС', pattern)
    pattern_1 = re.compile(r'ИБС')
    patt_2 = re.search(pattern_1, pattern)
    try:
        ihd_list.append(patt_2)
    except:
        pass  
# итого, обработаны все возможные паттерны    
    
ihd = []

for i in ihd_list:
    if not i:
        ihd.append(0)
    else:
        ihd.append(1)

## Поиск сахарного диабета

In [ ]:
dm_list = []
for sent in list_wo_punct:
    pattern = re.sub(r'сд', 'СД', sent )
    pattern = re.sub(r'сах.....', 'СД', pattern)
    pattern = re.sub(r'д.абет', 'СД', pattern)
    pattern_1 = re.compile(r'СД')
    patt_2 = re.search(pattern_1, pattern)
    try:
        dm_list.append(patt_2)
    except:
        pass  
    
dm_ = []

for i in dm_list:
    if not i:
        dm_.append(0)
    else:
        dm_.append(1)


## Поиск ожирения

In [ ]:
obes_list = []
for sent in list_wo_punct:
    pattern = re.sub(r'ожир.н..', 'ОЖИРЕНИЕ', sent )
    pattern = re.sub(r'.нжо.', 'ОЖИРЕНИЕ', pattern)
    
    #pattern = re.sub(r'торсков2', 'covid', pattern)
    pattern_1 = re.compile(r'ОЖИРЕНИЕ')
    patt_2 = re.search(pattern_1, pattern)
    #patt_2 = pattern_1.findall(''.join(pattern.split()))
    try:
        obes_list.append(patt_2)
    except:
        pass  
    
ob_ = []

for i in obes_list:
    if not i:
        ob_.append(0)
    else:
        ob_.append(1)

## Поиск обструктивной болезни лёгких

In [ ]:
copd_list = []
for sent in list_wo_punct:
    pattern = re.sub(r'хобл', 'ХОБЛ', sent )
    pattern = re.sub(r'другая обструктивная', 'ХОБЛ', pattern)
    pattern = re.sub(r'j44', 'ХОБЛ', pattern)
    pattern_1 = re.compile(r'ХОБЛ')
    patt_2 = re.search(pattern_1, pattern)
    try:
        copd_list.append(patt_2)
    except:
        pass  


copd_ = []
for i in copd_list:
    if not i:
        copd_.append(0)
    else:
        copd_.append(1)    

## Поиск бронхиальной астмы

In [ ]:
ba_list = []
for sent in list_wo_punct:
  pattern = re.sub(r'бронхиальная', 'БА', sent )
  pattern = re.sub(r'частично контролируемая', 'БА', pattern)
  pattern = re.sub(r'\sба\s', 'БА', pattern )
  pattern = re.sub(r'неконтролируемая', 'БА', pattern )
  pattern_1 = re.compile(r'БА')
  pattern_2 = re.search(pattern_1, pattern)
  try:
    ba_list.append(pattern_2)
  except:
    pass
ba = []
for i in ba_list:
    if not i:
        ba.append(0)
    else:
        ba.append(1)

## Определяем наличие дыхательной недостаточности у пациентов

In [ ]:
# нас интересует самый высокий уроведь ДН, т.к. у пациента при поступлении ДН может быть 2, а при выписке - 0,
# о тяжести пациента мы будем судить по наибольшему значению
rfl = []
for i in rf_:
    if i:
        rfl.append(max(np.float_(i)))
    else:
        rfl.append(0)

итого: по степеням дыхательной недостаточности мы имеем: {0.0: 1015, 1.0: 652, 2.0: 535, 3.0: 22}

## Определение тяжести

Пневмонии классифицируются на тяжёлые и нетяжёлые

In [ ]:
gr_list = []
for sent in list_wo_punct:
  pattern = re.sub(r'среднетяж.л..', '', sent)
  pattern = re.sub(r'ср.тяж.', '', sent)
  pattern = re.sub(r'нетяж.л..', '', pattern)
  pattern_1 = re.compile(r'тяж.л..')
  pattern_2 = re.search(pattern_1, pattern)
  try:
    gr_list.append(pattern_2)
  except:
    pass

In [ ]:
g = []
for i in gr_list:
    if not i:
        g.append(0)
    else:
        g.append(1)

## 2) Работа с лечением. 

In [ ]:
# задаём словарь, в котором ключ - истинное название препарата, а значения - варианты названий ( коммерческих) и
# названия с ошибками для обработки с помощью регулярных выражений

dict_treatment = {'лизиноприл':'л.з.....ил|лизитар|лизинеоприл', 
                  'цефтриаксон':'три.....ф|цефт.....он|цефтриакосн', 
                   "метопролол":"мет.....ол|эгилок", 
                  "фраксипарин":"фра......ин",
                  'тейкопланин': "тей......ин.",
                  "амброксол": "аброкол|амброко.|амб......|амброксолс\
                      |амброкосл|амброксолл|амброксолл|аброксол|амброксолл", 
                  "моксифлоксацин": 'пле....кс.|мокс.......цин',
                  "молсидомин":"м.лсид...н",
                  "спиронолактон":"сп.р.......он.|верош....н",
                 "бисопролол":"б.с.пр..ол|бисопралдол|бикард|конкор", 
                  "лансопрозол": "лан......ол|ланс.зол",
                 "меропенем":"мер....ем",
                  "аторвастатин": "липромак|аторв.......",
                 "дексаметазон": "декс.......н|дексаетазон",
                 "левофлоксацин": "лев.......цин",
                 "омепразол": "омез|омепр..ол",
                 "азитромицин": "азит.......",
                 "кандесартан": "к.с.рк",
                 "амоксициллин": "амоксицилин|амо........н",
                 "озельтамивир": "флустоп|озел.......р",
                 "эноксапарин": "клексан|кл...ан",
                 "имепенем": "им....ем",
                 "клопидогрел": "клопидогрель|плавикс|клорпидогрель",
                 "аспикард": "аспкиард|кардиомагнил|ас....рд",
                 "амлодипин":"амл....ин",
                 "амикацин": "ам.к...н",
                 "индапамид": "индап|инд.....д",
                 "беродуал": "бер.дуал|ипратропия.бромид",
                 "ацетилцистеин": "ацецемед|асс|ацц|ацемед|ацецезон",
                 "парацетамол": "парац.....л",
                 "гидроксихлорохин": "плаквенил|имар.|иммар.",
                 "метилпреднизолон": "медрол|м.дрол|метипред|диметилпреднизолон",
                 "теофиллин": "теофилин|теотард|теофилл",
                 "кларитромицин": "кларибацин|кла.......цин",
                 "лозартан": "лазртан|л...ртан|л.риста",
                 "валсартан": "валозартан|валз|вал....ан",
                 "моксонидин": "м.кс.н.дин",
                 "амиодарон": "ами.д.р.н|к.рд.рон",
                 "полимиксин": "колистин|колистат|к.л.стат",
                 "преднизолон": "преднезолон|пре.н.з.лон",
                 "рамиприл": "рам.л...|рам.прил",
                 "карбомазепин": "карб.м.з.п.н",
                 "кетаролак": "кет.р.лак|кет.рол|кет.н.в",
                 "гентамицин": "г.нт.мицин",
                 "цефепим": "ц.ф.пим",
                 "цефоперазон/сульбактам": "цеф.пер.зон|цеф.пер.зон.су.......",
                 "тайгециклин": "тиг.циклин|та.г.циклин",
                 "зопиклон": "з.п.клон|соне.|соне.с"}

cleaned_treatment = []
for words in tr:
    try:
        for i, j in dict_treatment.items():
            words = re.sub(j, i, words.lower())

        cleaned_treatment.append(words)
    except:
        cleaned_treatment.append("None")

На выходе мы получили список с правильными названиями препаратов. Если нет - возвращаем "None"

# 3) Работа с анализами

Мотивировочная часть:
Прокальцитонин - показатель, который обладает довольно выской чувствительностью и специфичностью к наличию у пациента батериальной инфекции, т.о., если уровень прокальцитонина в пределах референтных значений - антибиотики пациенту не показаны. 
Таким образом, мы хотим знать уровень прокальцитонина у пациента при поступлении и максимальный прокальцитонин (нужны ли были ему антибиотики за всё время пребывания в больнице)

In [ ]:
initial_pct = []
max_pct = []
try:
    for pct in pct_:
        if pct != 'None':
            initial_pct.append(pct[0])
            max_pct.append(max(pct))
        else:
            initial_pct.append(0)
            max_pct.append(0)
            
except:
    initial_pct.append(0)
    max_pct.append(0)


## Показанием для антибиотикотерапии является уровень прокальцитонина более 0.5. Определим этот признак. Однако, в нашем случае, я определяю как показание к антибактериальной терапии уровень выше 0,4, т.к. хочу минимизировать риск "пропустить" пациента. По некоторым данным 0.4 - "серая" зона, где уже может быть вероятность показаний к АБ-терапии

In [ ]:
ind_4_ab = []
for pct_i  in  max_pct_:
    if pct_i > .4:
        ind_4_ab.append(1)
    else:
        ind_4_ab.append(0)

Отлично, проблема решена

## Поработаем с остальными анализами

In [ ]:
# АСТ - неспецифический маркер повреждения печени

max_ast = []
try:
    for ast in ast_:
        if ast != 'None':
            max_ast.append(max(ast))
        else:
            max_ast.append(0)
except:
    max_ast.append(0)

# АЛТ - неспецифический маркер повреждения печени
max_alt = []
try:
    for alt in alt_:
        if alt != 'None':
            max_alt.append(max(alt))
        else:
            max_alt.append(0)
except:
    max_alt.append(0)



# ЛДГ - неспецифический маркер повреждения лёгких, повышение характерно именно
# для коронавирусной инфекции
initial_ldh = []
max_ldh = []
try:
    for ldh in ldh_:
        if  ldh == 'None':
            initial_ldh.append(0)
            max_ldh.append(0)
            
        else:
          try:
            initial_ldh.append(ldh[0])
            max_ldh.append(max(ldh))
          except:
            initial_ldh.append(0)
            max_ldh.append(0)            
except:
    initial_ldh.append(0)
    max_ldh.append(0)


# уровень гемоглобина 
max_hgb = []
try:
    for hgb in hgb_:
        if hgb != 'None':
            max_hgb.append(max(hgb))
        else:
            max_hgb.append(0)
except:
    max_hgb.append(0)


# креатинин - продукт обмена, который характеризует функцию почек

max_cre = []
min_cre = []
try:
    for cre in crea_:
        if cre != 'None':
            min_cre.append(min(cre))
            max_cre.append(max(cre))
        else:
            min_cre.append(0)
            max_cre.append(0)
except:
    min_cre.append(0)
    max_cre.append(0)


# СРБ - маркер воспаления, невоспалительный, при коронавирусе часто повышается
initial_crp = []
max_crp = []
try:
    for crp in crp_:
        if crp != 'None':
            initial_crp.append(crp[0])
            max_crp.append(max(crp))
        else:
            initial_crp.append(0)
            max_crp.append(0)
            
except:
    initial_crp.append(0)
    max_crp.append(0)


# уровень тромбоцитов

min_plt = []
max_plt = []
try:
    for plt in plt_:
        if plt != 'None':
            min_plt.append(min(plt))
            max_plt.append(max(plt))
        else:
            min_plt.append(0)
            max_plt.append(0)
except:
    max_plt.append(0)
    min_plt.append(0)


# уровень лейкоцитов
min_wbc = []
max_wbc = []
try:
    for wbc in wbc_:
        if wbc != 'None':
            min_wbc.append(min(wbc))
            max_wbc.append(max(wbc))
        else:
            min_wbc.append(0)
            max_wbc.append(0)
except:
    max_wbc.append(0)
    min_wbc.append(0)



# уровень СОЭ
initial_esr = []
max_esr = []
try:
    for esr in esr_:
        if esr != 'None':
          initial_esr.append(esr[0])
          max_esr.append(max(esr))
        else:
          initial_esr.append(0)
          max_esr.append(0)
            
except:
    initial_esr.append(0)
    max_esr.append(0)


# фибриноген - белок свёртывающей системы крови, может быть как маркер воспаления
initial_fibin = []
max_fibin = []
try:
    for fibin in fibrin_:
        if  fibin == 'None':
            initial_fibin.append(0)
            max_fibin.append(0)
            
        else:
          try:
            initial_fibin.append(fibin[0])
            max_fibin.append(max(fibin))
          except:
            initial_fibin.append(0)
            max_fibin.append(0)            
except:
    initial_fibin.append(0)
    max_fibin.append(0)

In [ ]:
# Собираем дата фрейм. Сначал в словарь

data_dct = {'ID':ID_, "Max_Procalcitonine": max_pct_,'Max_Platlets':max_plt, 
            "Min_Platlets":min_plt, "Init_ESR": initial_esr,
            "Max_ESR": max_esr, "Init_Fibrinogen": initial_fibin,
            "Max_Fibrinogen": max_fibin, "Initial_LDH": initial_ldh,
            "Max_LDH": max_ldh,
            'Max_Leucocytes': max_wbc,
            'Min_Leucocytes': min_wbc,'Hemoglobin': hgb_,
           'Max_Creatinine': max_cre, "Min_Cratinine": min_cre , 
            'Gender':sex_, 
            'initial_CRP':initial_crp,
            "Max_CRP":max_crp, 
            'Treatment': cleaned_treatment, 'Birthday': bir, 'Admission':ad,
            'Discharge':dis,  'Diagnosis': list_wo_punct,
            'COVID-19': c, 'Pneumonia': p, "Severity": g,
            "IHD": ihd, "COPD": copd_, "Bronchial_asthma": ba,
            "Obesity":ob_, "Diabetes": dm_, 'RF': rfl, "ALT": max_alt,
            "AST": max_ast, 'Ab_ind': ind_4_ab
           }
# затем пандас-датафрейм
dataFrame = pd.DataFrame(data_dct)

In [ ]:
# переводим наши даты в формат пандас
for i in ['Birthday', 'Admission', 'Discharge']:
  dataFrame[i] = pd.to_datetime(dataFrame[i], format = "%d:%m:%Y")
# длительность госпитализации = выписка - поступление
dataFrame["duration"] = (dataFrame['Discharge'] -
                         dataFrame['Admission']).dt.days

# возраст пациента рассчитаем: день рождения - поступление = 
#дни и целочисленно делим на 356
dataFrame["Age"] = ((dataFrame['Admission'] - 
                     dataFrame['Birthday']).dt.days) // 356
dataFrame = dataFrame.drop(['Birthday', 'Admission', 'Discharge'], axis=1)


# # т.к. в наших данных есть параметр "креатинин", для того, 
#чтобы улучшить данные, давайте высчитаем скорость клубочковой
# фильтрации. Однако, для начала надо перевести мкмоль 
#на литр в милиграммы на децилитр по формуле
# мкмоль на литр / 113
dataFrame['Max_Creatinine'] = dataFrame['Max_Creatinine'] / 113 


# т.к. в наших данных есть параметр "креатинин", для того, чтобы улучшить
# данные, давайте высчитаем скорость клубочковой
# фильтрации. СКФ обладает значительно более высокой диагностической ценностью
"""Levey AS, Stevens LA, Schmid CH, et al. for the CKD-EPI 
(Chronic Kidney Disease Epidemiology Collaboration). 
A New Equation to Estimate Glomerular Filtration Rate.
 Ann Intern Med. 2009 May 5; 150:604-612. PubMed ID: 19414839"""

if dataFrame.Gender.all() == 'male':
    if dataFrame.Max_Creatinine_mg_dl.all() < .9:
        dataFrame['GFR'] = 141 * (((dataFrame['Max_Creatinine_mg_dl'] /
                                  0.9)) ** (-0.411)) * 0.993 ** dataFrame['Age']
    else:
        dataFrame['GFR'] = 141 * (((hf['Max_Creatinine_mg_dl'] /
                                  0.9)) ** (-1.209)) * 0.993 ** dataFrame['Age']
else:
    if dataFrame.Max_Creatinine_mg_dl.all() < .7:
        dataFrame['GFR'] = 144 * (((dataFrame['Max_Creatinine_mg_dl'] / 
                                  0.7)) ** (-0.329)) * 0.993 ** dataFrame['Age']
    else:
        dataFrame['GFR'] = 144 * (((dataFrame['Max_Creatinine_mg_dl'] / 
                                  0.7)) ** (-1.209)) * 0.993 ** dataFrame['Age']


# также соотношение лейкоцитов к тромбоцитам может быть полезным
dataFrame["L/Pl_Ratio"] = dataFrame['Max_Leucocytes']
                                         / dataFrame['Max_Platlets'] * 100

In [ ]:
# оставляем только ковид и где есть прокальцитонин
covid_data = dataFrame[(dataFrame['COVID-19'] == 1) &
                      (dataFrame['Max_Procalcitonine'] > 0)]

In [ ]:
# доп. очистка данных. Номер эпикриза, уровень гемоглобина, макс.тромб., текстовые данные,
# минимальный уровень креатинина, ковид и пневмония не понадобятся для дальнейшего анализа
# т.к. они не отражают сути процесса и могут ввести в заблуждение
covid_data = covid_data.drop([ 'ID','Hemoglobin',
              'Diagnosis', 'Treatment', 'Min_Cratinine', 
              'COVID-19', 'Pneumonia', "Max_Platlets",
              'Init_Fibrinogen', 'Initial_LDH','Min_Leucocytes','initial_CRP',
              'Init_ESR' ], axis = 1)

# пол переведём в числовую матрицу
covid_data['Gender'] = covid_data['Gender'].map({'male':1, 'female': 0})

In [ ]:
# хотя и уровень дыхательной недостаточности определён цифрой, но это 
# по-сути категориальная колонка
covid_data["RF"] = covid_data["RF"].astype('category')

In [ ]:
# получим даммис от всех категориальных данных
covid_data = pd.get_dummies(covid_data)

In [ ]:
covid_data.shape

(863, 25)

## Итого мы имеем матрицу размерности 863 на 25

In [ ]:
covid_data.head()

,Min_Platlets,Max_ESR,Max_Fibrinogen,Max_LDH,Max_Leucocytes,Gender,Max_CRP,Severity,IHD,COPD,Bronchial_asthma,Obesity,Diabetes,ALT,AST,duration,Age,Max_Creatinine,RF_0.0,RF_1.0,RF_2.0,RF_3.0,GFR,Ab_ind,L/Pl_Ratio
0,142.8,54.0,4.8,513.0,10.0,1,88.7,0,0,1,0,0,0,57.26,42.59,25,58,0.94,0,0,0,0,87.0,0,38.1
1,261.5,36.0,10.0,287.6,8.3,0,47.4,1,0,0,0,0,0,101.68,59.71,18,54,0.87,0,0,0,0,92.0,0,20.2
2,225.4,36.0,9.3,851.4,12.5,0,57.4,0,0,0,0,0,0,69.51,35.10,16,54,0.76,0,0,0,0,96.0,0,29.0
3,323.3,57.0,3.7,690.3,10.9,0,116.0,0,0,0,0,0,0,46.83,44.42,29,48,0.91,0,0,1,0,94.0,1,19.5
4,168.8,52.0,7.3,567.1,7.7,0,189.6,0,0,0,0,0,0,43.23,35.31,23,57,0.87,0,0,0,0,90.0,1,33.1


In [ ]:
covid_data.to_csv('covid_data.csv')